## 1)

In [35]:
import urllib
import requests
from bs4 import BeautifulSoup
import re

link = "https://www.weforum.org/agenda/2020/03/covid-19-your-questions-answered/"
page = urllib.request.urlopen(link)
soup = BeautifulSoup(page, 'lxml')

ques_heading = soup.findAll('h2',attrs={"class":"st__content-block st__content-block--heading st__content-block--heading-1"})
ques1 = []
for elem in ques_heading :
    i = re.search("<b>.*</b>",str(elem)).span()[0]
    f = re.search("<b>.*</b>",str(elem)).span()[1]
    ques1.append(str(elem)[i+3 : f - 4])



answer_heading = soup.findAll('div',attrs={"class":"st__content-block st__content-block--text"})
answer1 = []
for elem in answer_heading :
    i = re.search("<p>.*</p>",str(elem)).span()[0]
    f = re.search("<p>.*</p>",str(elem)).span()[1]
    answer1.append(str(elem)[i+3 : f - 4])



In [36]:
print(ques1)

['Can you have COVID-19 and show no symptoms? ', 'Can one get coronavirus for a second time? ', 'How long can the virus survive on surfaces? ', 'How has attention to other major illnesses (like cancer) been affected? ', 'Is it possible that one day this will happen again? ', 'Is there any vaccine for this virus? ']


In [37]:
#answer1 = [ans for ans in answer1 if 'https://' not in ans]
#answer1 = [ans for ans in answer1 if not ans.startswith('<i>')]

In [38]:
ques1

['Can you have COVID-19 and show no symptoms? ',
 'Can one get coronavirus for a second time? ',
 'How long can the virus survive on surfaces? ',
 'How has attention to other major illnesses (like cancer) been affected? ',
 'Is it possible that one day this will happen again? ',
 'Is there any vaccine for this virus? ']

In [39]:
filtered_ans1 = []
for i,j in enumerate(answer1):
    if i in [1,5,9,12,15,21]:
        j = re.sub('<.*>',"", j)
        filtered_ans1.append(j)

In [40]:
len(filtered_ans1)

6

In [19]:
t = """
'This depends on the surface material and the amount of virus present. <a href="https://www.nejm.org/doi/full/10.1056/NEJMc2004973" rel="noopener" target="_blank">A recent study</a> investigated the persistence of coronavirus on stainless steel, plastic, cardboard and copper. The authors used an initial virus concentration similar to that found in the respiratory tract of infected people. The virus remained detectable for between 10 hours and three days, with the shortest persistence on copper and the longest on plastic. <a href="https://www.medrxiv.org/content/10.1101/2020.03.05.20030502v1.full.pdf+html" rel="noopener" target="_blank">Another study</a> that looked at the related virus SARS-CoV - the virus responsible for the SARS outbreak in 2003 - found it was detectable for up to nine days. However, just because a virus is detectable does not mean it poses a health risk. '
"""

# 2)

In [29]:
import urllib
import requests
from bs4 import BeautifulSoup


link = "http://www.emro.who.int/health-topics/corona-virus/questions-and-answers.html"
page = urllib.request.urlopen(link)
soup = BeautifulSoup(page, 'lxml')

heading = soup.findAll('div',attrs={"class":"article-content"})





ques2 = []
for elem in re.findall("<h3>.*</h3>",str(heading))[1:]:
    print(elem)
    i = re.search("<h3>.*</h3>",str(elem)).span()[0]
    f = re.search("<h3>.*</h3>",str(elem)).span()[1]
    con = elem[i+4 : f - 5]
    ques2.append(con)

answer2 = []
l = len(re.findall("<p>.*</p>",str(heading)))

for elem in re.findall("<p>.*</p>",str(heading))[0:l-2]:
    print(elem)
    i = re.search("<p>.*</p>",str(elem)).span()[0]
    f = re.search("<p>.*</p>",str(elem)).span()[1]
    con = elem[i+3 : f - 4]
    answer2.append(con)
    

ques2.remove(ques2[-1])
answer2.remove(answer2[-1])

<h3>What is a coronavirus?</h3>
<h3>What is a novel coronavirus?</h3>
<h3>Can humans become infected with a novel coronavirus of animal source?</h3>
<h3>What are the symptoms of someone infected with a coronavirus?</h3>
<h3>Can coronaviruses be transmitted from person to person?</h3>
<h3>Is there a vaccine for a novel coronavirus?</h3>
<h3>Is there a treatment for a novel coronavirus?</h3>
<h3>What can I do to protect myself?</h3>
<h3>Are health workers at risk from a novel coronavirus?</h3>
<h3>What WHO recommendations for countries?</h3>
<h3>Where can I find more information about known coronaviruses?</h3>
<p>Coronaviruses are a large family of viruses that are known to cause illness ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).</p>
<p>A novel coronavirus (CoV) is a new strain of coronavirus that has not been previously identified in humans.</p>
<p>Detailed investigations found that S

In [50]:
tot_ques = ques1 + ques2
tot_ans = filtered_ans1 + answer2

In [49]:
len(ques2), len(answer2)

(10, 10)

In [31]:
import pandas as pd

FAQ = pd.read_excel('C:/Users/ABHISHEK DUBEY/Documents/Abhishek/Resources/AnythingButML/chatbot/data/FAQ.xlsx')

In [51]:
temp = pd.DataFrame({'Question' : tot_ques, 'Answer' : tot_ans})

In [52]:
FAQ = pd.concat([FAQ, temp], axis = 0)

In [53]:
FAQ.to_excel('FAQ.xlsx', index = False)


In [56]:
import pandas as pd
import requests
import json

url1 = f'https://api.covid19india.org/data.json'
url2 = f'https://api.covid19india.org/v2/state_district_wise.json'

response1 = requests.get(url1)
response1_dict = json.loads(response1.text)

response2 = requests.get(url2)
response2_dict = json.loads(response2.text)

final_state_list = []
final_district_list = []
valid_distrcit_key = ['district','confirmed']
valid_state_key = ['active','recovered','state','deaths','confirmed']

for state_dict in response1_dict['statewise']:
    temp = (list(map(lambda key,value :  key+"#"+value if key in valid_state_key else None , state_dict.keys(),state_dict.values() )))
    final_state_list.append(" ".join(list(filter(None, temp))) )

for response in response2_dict:
    state = response['state']
    for district in response['districtData']:
        temp = (list(map(lambda key,value :  key+"#"+str(value) if key in valid_distrcit_key else None  , district.keys(),district.values())))
        final_district_list.append(" ".join(list(filter(None, temp))))

state_dataframe = pd.DataFrame({'Question' : final_state_list, 'Answer' : final_state_list})
district_dataframe = pd.DataFrame({'Question':final_district_list,'Answer':final_district_list})
final_dataframe = pd.concat([state_dataframe,district_dataframe])
final_dataframe.to_excel('Data.xlsx',index=False);